In [65]:
from time import time
from scipy.sparse import csc_matrix
import tensorflow as tf
import keras
import numpy as np
import h5py
import pandas as pd
print("Tensorflow verison:", tf.__version__)


Tensorflow verison: 2.15.0


In [42]:

def load_data_1m(path='./', delimiter='::', frac=0.1, seed=1234):

    tic = time()
    print('reading data...')
    data = np.loadtxt(path+'movielens_1m_dataset.dat', skiprows=0, delimiter=delimiter).astype('int32')
    print('taken', time() - tic, 'seconds')

    n_u = np.unique(data[:,0]).size  # num of users
    n_m = np.unique(data[:,1]).size  # num of movies
    n_r = data.shape[0]  # num of ratings

    udict = {}
    for i, u in enumerate(np.unique(data[:,0]).tolist()):
        udict[u] = i
    mdict = {}
    for i, m in enumerate(np.unique(data[:,1]).tolist()):
        mdict[m] = i

    np.random.seed(seed)
    idx = np.arange(n_r)
    np.random.shuffle(idx)

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_r):
        u_id = data[idx[i], 0]
        m_id = data[idx[i], 1]
        r = data[idx[i], 2]

        if i < int(frac * n_r):
            test_r[mdict[m_id], udict[u_id]] = r
        else:
            train_r[mdict[m_id], udict[u_id]] = r

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_r - int(frac * n_r)))
    print('num of test ratings: {}'.format(int(frac * n_r)))

    return n_m, n_u, train_r, train_m, test_r, test_m

In [43]:
path =  '../data/movielens_1M/'
n_m, n_u, train_r, train_m, test_r, test_m = load_data_1m(path=path, delimiter=':', frac=0.1, seed=1234)

reading data...
taken 0.7960717678070068 seconds
data matrix loaded
num of users: 6040
num of movies: 3706
num of training ratings: 900189
num of test ratings: 100020


# Hyperparameter Settings

In [44]:
# Common hyperparameter settings
n_hid = 500
n_dim = 5
n_layers = 2
gk_size = 3
lambda_2 = 70.
lambda_s = 0.018
iter_p = 50
iter_f = 10
epoch_p = 20
epoch_f = 30
dot_scale = 0.5

In [45]:
tf.compat.v1.disable_eager_execution()
R = tf.compat.v1.placeholder("float", [n_m, n_u])

# Network Function

In [46]:
def local_kernel(u, v):
    
    dist = tf.norm(u - v, ord=2, axis=2)
    hat = tf.maximum(0., 1. - dist**2)

    return hat

In [70]:
def kernel_layer(x, n_hid=n_hid, n_dim=n_dim, activation=tf.nn.sigmoid, lambda_s=lambda_s, lambda_2=lambda_2, name=''):
    
    with tf.compat.v1.variable_scope(name, reuse=tf.compat.v1.AUTO_REUSE):
        W = tf.compat.v1.get_variable('W', [x.shape[1], n_hid])
        n_in = x.get_shape().as_list()[1]
        u = tf.compat.v1.get_variable('u', initializer=tf.random.truncated_normal([n_in, 1, n_dim], 0., 1e-3))
        v = tf.compat.v1.get_variable('v', initializer=tf.random.truncated_normal([1, n_hid, n_dim], 0., 1e-3))
        b = tf.compat.v1.get_variable('b', [n_hid])

    w_hat = local_kernel(u, v)
    
    sparse_reg = keras.regularizers.l2(lambda_s)
    sparse_reg_term = keras.regularizers(sparse_reg, [w_hat])
    
    l2_reg = keras.regularizers.l2(lambda_2)
    l2_reg_term = tf.keras.regularizers(l2_reg, [W])

    W_eff = W * w_hat  # Local kernelised weight matrix
    y = tf.matmul(x, W_eff) + b
    y = activation(y)

    return y, sparse_reg_term + l2_reg_term

In [69]:
keras.regularizers.Regularizer(0.018)

TypeError: Regularizer() takes no arguments

In [53]:
def global_kernel(input, gk_size, dot_scale):
    
    avg_pooling = tf.reduce_mean(input, axis=1)  # Item (axis=1) based average pooling
    avg_pooling = tf.reshape(avg_pooling, [1, -1])
    n_kernel = avg_pooling.shape[1].value

    conv_kernel = tf.compat.v1.get_variable('conv_kernel', initializer=tf.random.truncated_normal([n_kernel, gk_size**2], stddev=0.1))
    gk = tf.matmul(avg_pooling, conv_kernel) * dot_scale  # Scaled dot product
    gk = tf.reshape(gk, [gk_size, gk_size, 1, 1])

    return gk

In [54]:
def global_conv(input, W):
    
    input = tf.reshape(input, [1, input.shape[0], input.shape[1], 1])
    conv2d = tf.nn.relu(tf.nn.conv2d(input, W, strides=[1,1,1,1], padding='SAME'))

    return tf.reshape(conv2d, [conv2d.shape[1], conv2d.shape[2]])

In [71]:
y = R
reg_losses = None

for i in range(n_layers):
    y, reg_loss = kernel_layer(y, name=str(i))
    reg_losses = reg_loss if reg_losses is None else reg_losses + reg_loss

pred_p, reg_loss = kernel_layer(y, n_u, activation=tf.identity, name='out')
reg_losses = reg_losses + reg_loss

# L2 loss
diff = train_m * (train_r - pred_p)
sqE = tf.nn.l2_loss(diff)
loss_p = sqE + reg_losses

optimizer_p = tf.optimizers.ScipyOptimizerInterface(loss_p, options={'disp': True, 'maxiter': iter_p, 'maxcor': 10}, method='L-BFGS-B')
     

<tf.Variable '0/u:0' shape=(6040, 1, 5) dtype=float32>


TypeError: 'module' object is not callable

In [61]:
import tensorflow as tf

# Tạo một đối tượng ScipyOptimizerInterface
optimizer = tf.optimizers.ScipyOptimizerInterface(
    optimizer="L-BFGS-B",
    method="L-BFGS-B",
    options={"maxiter": 100},
)

# Tối ưu hóa một hàm mục tiêu với các ràng buộc
def f(x):
    return x[0] ** 2 + x[1] ** 2

def g(x):
    return x[0] + x[1] - 1

x0 = [0.0, 0.0]
optimizer.minimize(f, x0, constraints=[g])

# Xuất ra giá trị tối ưu
print(x0)


AttributeError: module 'keras.api._v2.keras.optimizers' has no attribute 'ScipyOptimizerInterface'